In [2]:
!pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/fc/a5/4d82be566f069d7a9a702dcdf6f9106df0e0b042e738043c0cc7ddd7e3f6/pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/65/58/f9c9e6be752e9fcb8b6a0ee9fb87e6e7a1f6bcab2cdc73f02bb7ba91ada0/tzdata-2024.1-py2.py3-none-any.whl.metadata
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.0 MB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


In [3]:
import torch
import torch.nn as nn
import pandas as pd
import sklearn
import numpy
import torch.utils
from torch.utils.data import Dataset,DataLoader
import torch.utils.data
from tqdm.notebook import trange, tqdm
from statistics import mean, stdev, variance
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [4]:
class MLP_model (nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.input_fc=nn.Linear(input_dim,800)
        self.hidden_fc=nn.Linear(800,600)
        self.hidden_fc2=nn.Linear(600,400)
        self.hidden_fc3=nn.Linear(400,400)
        self.hidden_fc4=nn.Linear(400,400)
        self.hidden_fc5=nn.Linear(400,400)
        self.hidden_fc6=nn.Linear(400,400)
        self.hidden_fc7=nn.Linear(400,400)
        self.hidden_fc8=nn.Linear(400,400)
        self.hidden_fc9=nn.Linear(400,400)
        self.hidden_fc10=nn.Linear(400,100)

        self.output_fc=nn.Linear(100,output_dim)
        self.dropout = nn.Dropout(0.25)

    def forward(self,x):
       batch_size=x.shape[0]
       x = x.view(batch_size, -1)
       x=self.dropout(x)
       m_1=nn.functional.relu(self.input_fc(x))
       m_2=nn.functional.relu(self.hidden_fc(m_1))
       m_3=nn.functional.relu(self.hidden_fc2(m_2))
       m_4=nn.functional.relu(self.hidden_fc3(m_3))
       m_4=nn.functional.relu(self.hidden_fc4(m_4))
       m_4=nn.functional.relu(self.hidden_fc5(m_4))
       m_4=nn.functional.relu(self.hidden_fc6(m_4))
       m_4=nn.functional.relu(self.hidden_fc7(m_4))
       m_4=nn.functional.relu(self.hidden_fc8(m_4))
       m_4=nn.functional.relu(self.hidden_fc9(m_4))
       m_4=nn.functional.relu(self.hidden_fc10(m_4))
       m_4=self.dropout(m_4)

       #output=nn.functional.sigmoid(self.output_fc(m_4))
       output=(self.output_fc(m_4))

       return output 
    
""" 
class MLP_model (nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.input_fc=nn.Linear(input_dim,50)
        self.input_fc2=nn.Linear(50,25)
        self.output_fc=nn.Linear(25,output_dim)

    def forward(self,x):
       batch_size=x.shape[0]
       x = x.view(batch_size, -1)
       m_1=nn.functional.relu(self.input_fc(x))
       m_2=nn.functional.relu(self.input_fc2(m_1))

       output=self.output_fc(m_2)
       return output """
    


' \nclass MLP_model (nn.Module):\n    def __init__(self, input_dim, output_dim):\n        super().__init__()\n        self.input_fc=nn.Linear(input_dim,50)\n        self.input_fc2=nn.Linear(50,25)\n        self.output_fc=nn.Linear(25,output_dim)\n\n    def forward(self,x):\n       batch_size=x.shape[0]\n       x = x.view(batch_size, -1)\n       m_1=nn.functional.relu(self.input_fc(x))\n       m_2=nn.functional.relu(self.input_fc2(m_1))\n\n       output=self.output_fc(m_2)\n       return output '

In [5]:
def calculate_accuracy(y_pred, y):
  
    top_pred = y_pred.argmax(1)
    top_y= y.argmax(1)
    correct = top_pred.eq(top_y).sum()
    #print(top_pred)
    acc = correct.float() / y.shape[0]
    #print(correct)
    return acc

def model_train(model,dataloader,optimizer,citerion,device):
    epoch_loss=0
    epoch_acc=0
    model.train()

    for i,data in enumerate(dataloader):
        feature,label=data
        feature=feature.to(device)
        label=label.to(device)

        optimizer.zero_grad()
        y_pred=model(feature)
        loss=citerion(y_pred,label)
        acc=calculate_accuracy(y_pred,label)
        loss.backward()
        optimizer.step()

        epoch_loss+=loss.item()
        epoch_acc+=acc.item()

    return epoch_loss/len(dataloader),epoch_acc/len(dataloader)


In [6]:
def model_evaluate(model,dataloader,criterion,device):
    epoch_loss=0
    epoch_acc=0

    model.eval()
    with torch.no_grad():
        for i,data in enumerate(dataloader):
            feature,label=data
            feature=feature.to(device)
            label=label.to(device)

            y_pred=model(feature)

            loss=criterion(y_pred,label)
            acc=calculate_accuracy(y_pred,label)

            epoch_loss+=loss.item()
            epoch_acc+=acc.item()
    return epoch_loss/len(dataloader),epoch_acc/len(dataloader)


In [7]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(pd.__version__)
print(sklearn.__version__)

True
NVIDIA A100-SXM4-80GB
2.2.2
1.3.2


In [8]:
class train_dataset(Dataset):
    def __init__(self,csv_file_feature,csv_meta,training_target):
        inputdata=pd.read_csv(csv_file_feature)
        # Standardize the original data
        input_mean, input_std, input_var = inputdata.mean(), inputdata.std(), inputdata.var()
        new_std = 1
        # print("MEAN: \n", input_mean)
        # print("StandardDeviation: \n", input_std)
        # print("Variance: \n", input_var)
        self.base_frame=(inputdata-input_mean)/(input_std/new_std)
        # self.base_frame=inputdata
        self.meta_frame=pd.read_csv(csv_meta,dtype={'sample_name':'int32',
                      'apoe4':'string',
                      'bmi':'float32',
                      'diagnosis':'string',
                      'amlyoid_positive':'float32',
                      'visit':'int32'})
        self.target_frame=self.meta_frame[training_target]
        self.base_frame.drop(columns=['sample_name'])

    def __len__(self):
        return len(self.target_frame)
    
    def __getitem__(self, index):
        label=0
        if self.target_frame[index]=='positive':
            label=[1,0]
        else:
            label=[0,1]
        
        feature=self.base_frame.iloc[index]
        feature_tensor=torch.tensor(feature.values).float()

        label_tensor=torch.tensor(label).float()

        return feature_tensor,label_tensor


In [9]:
train_data=train_dataset('train_data.csv','Meta.csv','apoe4')
train_size= int(0.70*len(train_data))
test_size=len(train_data)-train_size

training_data,testing_data=torch.utils.data.random_split(train_data,[train_size,test_size])

# for ele in training_data:
#     print(ele)
# mean = torch.mean(training_data)
# mean

In [10]:
batch_size = 8
lr = 0.0001
EPOCHS = 50
n_splits = 5  # Number of folds



model=MLP_model(883,2) 

optimizer = torch.optim.Adam(model.parameters(),lr=lr)
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

training_dataloader=DataLoader(training_data,batch_size=8,shuffle=True)
testing_dataloader=DataLoader(testing_data,batch_size=8,shuffle=True)

# training_features = [data[0] for data in training_data]
# training_labels = [data[1]for data in training_data]
# training_feature_dataloader = DataLoader(training_features,batch_size=8,shuffle=True)
# training_label_dataloader = DataLoader(training_labels,batch_size=8,shuffle=True)

kf = KFold(n_splits=3, shuffle=True, random_state=42)

for epoch in range(EPOCHS):


    train_loss,train_acc=model_train(model,training_dataloader,optimizer,criterion,device)
    # scores = cross_val_score(model, training_feature_dataloader, training_label_dataloader, cv=kf)
    test_loss,test_acc=model_evaluate(model,testing_dataloader,criterion,device)
    print(f'\t E {epoch} Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t E {epoch} Test. Loss: {test_loss:.3f} |  Test. Acc: {test_acc*100:.2f}%')








	 E 0 Train Loss: 0.696 | Train Acc: 37.50%
	 E 0 Test. Loss: 0.691 |  Test. Acc: 62.50%
	 E 1 Train Loss: 0.689 | Train Acc: 68.18%
	 E 1 Test. Loss: 0.690 |  Test. Acc: 58.33%
	 E 2 Train Loss: 0.683 | Train Acc: 69.32%
	 E 2 Test. Loss: 0.691 |  Test. Acc: 54.17%
	 E 3 Train Loss: 0.672 | Train Acc: 69.32%
	 E 3 Test. Loss: 0.690 |  Test. Acc: 54.17%
	 E 4 Train Loss: 0.625 | Train Acc: 69.32%
	 E 4 Test. Loss: 0.724 |  Test. Acc: 54.17%
	 E 5 Train Loss: 0.559 | Train Acc: 69.32%
	 E 5 Test. Loss: 0.761 |  Test. Acc: 58.33%
	 E 6 Train Loss: 0.607 | Train Acc: 61.36%
	 E 6 Test. Loss: 0.730 |  Test. Acc: 54.17%
	 E 7 Train Loss: 0.597 | Train Acc: 61.36%
	 E 7 Test. Loss: 0.672 |  Test. Acc: 58.33%
	 E 8 Train Loss: 0.496 | Train Acc: 69.32%
	 E 8 Test. Loss: 0.712 |  Test. Acc: 58.33%
	 E 9 Train Loss: 0.425 | Train Acc: 69.32%
	 E 9 Test. Loss: 0.873 |  Test. Acc: 58.33%
	 E 10 Train Loss: 0.332 | Train Acc: 69.32%
	 E 10 Test. Loss: 1.065 |  Test. Acc: 62.50%
	 E 11 Train Loss: 

In [ ]:
# --normalize--: not really helpful with the original classifier
# Cross validation
# improvement options (low dep-high dim)
# transformer model (pytorch?)
# encoding methods

# 2217

In [ ]:
bio summer meeting: 6.19